# 1. 対話コーパスの前処理

    -データの読み込み
    -ルビ削除

In [1]:
import re

novels = ["gingatetsudono_yoru.txt", "serohikino_goshu.txt", "chumonno_oi_ryoriten.txt",
         "gusukobudorino_denki.txt", "kaeruno_gomugutsu.txt", "kaino_hi.txt", "kashiwabayashino_yoru.txt",
         "kazeno_matasaburo.txt", "kiirono_tomato.txt", "oinomorito_zarumori.txt"]  # 青空文庫より

text = ""
for novel in novels:
    with open("kenji_novels/"+novel, mode="r", encoding="utf-8") as f:  # ファイルの読み込み
        text_novel = f.read()
    text_novel = re.sub("《[^》]+》", "", text_novel)  # ルビの削除
    text_novel = re.sub("［[^］]+］", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("〔[^〕]+〕", "", text_novel)  # 読みの注意の削除
    text_novel = re.sub("[ 　\n「」『』（）｜※＊…]", "", text_novel)  # 全角半角スペース、改行、その他記号の削除
    text += text_novel

print("文字数:", len(text))
print(text)

FileNotFoundError: ignored

# 2. ひらがな変換

In [2]:
from pykakasi import kakasi

seperator = "。"  # 。をセパレータに指定
sentence_list = text.split(seperator)  # セパレーターを使って文章をリストに分割する
sentence_list.pop() # 最後の要素は空の文字列になるので、削除
sentence_list = [x+seperator for x in sentence_list]  # 文章の最後に。を追加

kakasi = kakasi()
kakasi.setMode("J", "H")  # J(漢字) からH(ひらがな)へ
conv = kakasi.getConverter()
for sentence in sentence_list:
    print(sentence)
    print(conv.do(sentence))
    print()

ModuleNotFoundError: ignored

In [3]:
text = text.replace("苹果", "ひょうか")

seperator = "。"
sentence_list = text.split(seperator) 
sentence_list.pop() 
sentence_list = [x+seperator for x in sentence_list]

for sentence in sentence_list:
    print(sentence)
    print(conv.do(sentence))
    print()

# 3. 文字一覧確認

In [4]:
kana_text = conv.do(text)  # 全体をひらがなに変換
print(set(kana_text))  # set()で文字の重複をなくす

NameError: ignored

# 4. テキストデータの保存

In [5]:
print(kana_text)
with open("kana_kenji.txt", mode="w", encoding="utf-8") as f:
    f.write(kana_text)

NameError: ignored

# 5. 対話の学習

In [6]:
import pickle

hiragana = "ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞ\
ただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽ\
まみむめもゃやゅゆょよらりるれろゎわゐゑをん"

katakana = "ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾ\
タダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポ\
マミムメモャヤュユョヨラリルレロヮワヰヱヲンヴ"

chars = hiragana + katakana

with open("kana_kenji.txt", mode="r", encoding="utf-8") as f:  # 前処理したファイル
    text = f.read()
    
for char in text:  # ひらがな、カタカナ以外でコーパスに使われている文字を追加
    if char not in chars:
        chars += char
        
chars += "\t\n"  # タブと改行を追加
        
chars_list = sorted(list(chars))  # 文字列をリストに変換してソートする
print(chars_list)

with open("kana_chars.pickle", mode="wb") as f:  # pickleで保存
    pickle.dump(chars_list, f)

FileNotFoundError: ignored

# 7. 文字のベクトル化

In [7]:
import numpy as np

# インデックスと文字で辞書を作成
char_indices = {}
for i, char in enumerate(chars_list):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars_list):
    indices_char[i] = char
    
n_char = len(chars_list)
max_length_x = 128

# 文章をone-hot表現に変換する関数
def sentence_to_vector(sentence):
    vector = np.zeros((1, max_length_x, n_char), dtype=np.bool)
    for j, char in enumerate(sentence):
        vector[0][j][char_indices[char]] = 1
    return vector

NameError: ignored

In [8]:
from keras.models import load_model

encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

def respond(input_data, beta=6):
    state_value = encoder_model.predict(input_data)
    y_decoder = np.zeros((1, 1, n_char))  # decoderの出力を格納する配列
    y_decoder[0][0][char_indices['\t']] = 1  # decoderの最初の入力はタブ。one-hot表現にする。

    respond_sentence = ""  # 返答の文字列
    while True:
        y, h = decoder_model.predict([y_decoder, state_value])
        p_power = y[0][0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power)) 
        next_char = indices_char[next_index]  # 次の文字
        
        if (next_char == "\n" or len(respond_sentence) >= max_length_x):
            break  # 次の文字が改行のとき、もしくは最大文字数を超えたときは終了
            
        respond_sentence += next_char
        y_decoder = np.zeros((1, 1, n_char))  # 次の時刻の入力
        y_decoder[0][0][next_index] = 1

        state_value = h  # 次の時刻の状態

    return respond_sentence

OSError: ignored

# 8. 対話の検証

In [ ]:
sentences = ["けんじさん、こんにちは。",
             "カムパネラってしってますか？",
             "ああ、おなかすいた。",
             "きょうはいいてんきですね。",
             "すきなたべものは、なんですか。",
             "きょうは、さんぽにいきました。",
             "パソコンかスマホは、つかったことありますか？"]

for sentence in sentences:
    vector = sentence_to_vector(sentence)
    print("Input:", sentence)
    print("Response", respond(vector))
    print()

Input: けんじさん、こんにちは。
Response せいねんはいいました。

Input: カムパネラってしってますか？
Response ジョバンニはまるであいさつして、そこでむねがいっぱいになっていました。

Input: ああ、おなかすいた。
Response みると、そのときは、あのかわりました。

Input: きょうはいいてんきですね。
Response それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それか

Input: すきなたべものは、なんですか。
Response といいました。

Input: きょうは、さんぽにいきました。
Response それから、そのとき、あかいはかせは、まるでいちめんにおおきなかおをみているのでした。

Input: パソコンかスマホは、つかったことありますか？
Response そうだ。



# 9. モデル同士の会話

In [ ]:
response_a = "こんにちは。"
response_b = ""
for i in range(100):
    print("賢治A:", response_a)
    print()  
    vector_a = sentence_to_vector(response_a)
    
    response_b = respond(vector_a)
    print("賢治B:", response_b)
    print()
    vector_b = sentence_to_vector(response_b)
    
    response_a = respond(vector_b)

賢治A: こんにちは。

賢治B: はいはいいね。

賢治A: はやくはなんだか。

賢治B: はやしのなかではない。

賢治A: だいへんきょうしつのなかではないんだ。

賢治B: そのときぼくはもうあのきりのようなものがあるんだ。

賢治A: そしてまたさぶろうはすこしおおきなこえでした。

賢治B: それから、そのとき、あかいはかせは、あかいはかせは、そのとき、きつねがいいました。

賢治A: そしてしばらくあおくひきはじめました。

賢治B: そしてそのにんは、そのとき、それから、そのとき、あなたのはらのうちにはながれていました。

賢治A: そしてそのこがありました。

賢治B: そのにんは、そのとき、ちょうどあのときは、あのかりのひとつのおおきなおおきなひとりのしたにはながら、そのとき、そのとき、ちょうどそのとき、あなたのはらのはなし、そのとき、それから、そのとき、きっとまっくろなのはらのおとこのこがすこしかおをあかくしながら、そ

賢治A: といいました。

賢治B: そしてそのときは、あかいはかせは、あかいひをついていきました。

賢治A: そしてそのときは、あかいはこのときあるいて、あかいはかせは、いちめんにさわって、そのおとこは、そのときあたまをかけて、それからそのままのところへいってしまいました。

賢治B: そしてそのときは、あかいはかせは、そのあおいかみのように、おおきなこえでした。

賢治A: ところがそのときは、あかいはかせは、そのときは、あかいはかせのまえにおおきなこえでした。

賢治B: そしてそのまんなかにはいってきたのです。

賢治A: かぜのまたさぶろうはまたさぶろうはまたおとうさんのところへいってしまいました。

賢治B: みんなは、そのときは、あおいひかりをして、それから、そのままのそとをみていましたが、そのとき、おかあさんは、あのかわりました。

賢治A: それはだいじょうぶです。

賢治B: けれども、いつかではいってきて、そのとき、きっとまたいっしょうけんいのちになっているんだ。

賢治A: それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから？それから

賢治B: そしてそのおとこは、もうあのきりのようになって、そのときは、あかいはあのきりのなかにはいってみました。

賢治A: そしてそのとき、あかいはかせは、そのとき、おおきなこえでした。

賢治B: そのとき、そのとき、そのときは、あるいて、そのとき、ちょうどそのとき、ちょうどおまえたちのほうでは、あのとりのこのなかにはいっているのでした。

賢治A: そしてそのまえにちょうどそのとき、それから、そのとき、あかいのはらには、あのとりのきのしたにはながら、そのとき、あかいはかせは、そのばんのあかりをみていました。

賢治B: そしてそのときは、あかいはかせは、そのとき、ちょうどそのときは、あかいはかせのまえにおおきなきがするのでした。

賢治A: そしてそのとき、そのとき、あかいはかせは、そのばんのさんかくしるべのあかりを、あかいはあのあかりをつかまえました。

賢治B: そしてそのにんは、ちょうどそのとき、ブドリは、そのとき、きれいなかおをして、そのときあしをたてて、そのときあたまをひとつとりました。

賢治A: そしてそのおとこは、きっとみんなはなにかしわのきがいっぱいになりました。

賢治B: そのとき、ちょうどあのとりのところには、あかいはあのあかりのようなかたちをして、そのとき、きつねのあかりが、まるでいちめんにきたのです。

賢治A: そしてそのこはあしをあけて、いっしょうけんいのち、あなたのはらにはいっていました。

賢治B: そしてそのときは、あかいはかせは、そのかわのみずのなかにはいってきました。

賢治A: そしてそのにんは、あかいのはらのうちにはいっていました。

賢治B: そのとき、あかいはこんなにんに、そのとき、あかいのことは、あかいのはらのはなしのようになって、そのとき、そこでしたのです。

賢治A: そしてそのにんは、そのとききしゃのきりのようになって、そのときは、ああ、あかいはいっていました。

賢治B: そしてそのおとこは、なにかのかんそくのはらをみていましたが、そのとき、それから、どうもありませんでした。

賢治A: そしてそのおとこは、きっとまっていました。

賢治B: そしてそのこは、おおきなこえでした。

賢治A: そのとき、あのかわりは、おおきなきがするのでした。

賢治B: そしてそのこがいいました。

賢治A: そのと